In [2]:
import pandas as pds

In [3]:
import requests
domain = "ajc.com"

def get_pagerank(domain):
    headers = {'API-OPR':'4ok88wgckg8o0cgcswo4gc4kkc0sgocw0woww4o0'}
    url = 'https://openpagerank.com/api/v1.0/getPageRank?domains%5B0%5D=' + domain
    request = requests.get(url, headers=headers)
    if request.status_code == 200:
        result = request.json()
        if result['status_code'] == 200:
            pr_int = result['response'][0]['page_rank_integer']
            pr_dec = result['response'][0]['page_rank_decimal']
            rank = result['response'][0]['rank']

            return pr_int, pr_dec, rank



In [4]:

import whois

def whois_checker(domain):
    w = whois.whois(domain)
    domain_registrar = w.get('registrar')
    postal_code = w.get('registrant_postal_code')
    country = w.get('country')

    return  domain_registrar, postal_code, country



In [5]:
import virustotal_python
from pprint import pprint
from base64 import urlsafe_b64encode
import json


def virus_scan(domain):
    with virustotal_python.Virustotal("242305dedab935c6a39736d0749ae88f2b68f1e2f0c8e82c81d8955c5fd194d3") as vtotal:
        resp = vtotal.request(f"domains/{domain}")
        virus_data = resp.json()['data']['attributes']['last_analysis_stats']
        return virus_data['harmless'], virus_data['malicious'], virus_data['suspicious'], virus_data['undetected'], virus_data['timeout']





In [6]:
from requests_html import HTMLSession
from collections import Counter
from urllib.parse import urlparse

#try except outside this func
def link_checker(domain):
    session = HTMLSession()

    r = session.get("https://"+domain)
    unique_netlocs = Counter(urlparse(link).netloc for link in r.html.absolute_links)
    outbound_n = 0
    local_n = 0
    links = []
    for link in unique_netlocs:
        # print(link, unique_netlocs[link])
        links.append(link)
        if domain.lower() in link:
            local_n += unique_netlocs[link]
        else:
            outbound_n += unique_netlocs[link]

    return local_n, outbound_n, local_n + outbound_n, links

# local_n, outbound_links, total_links, links = link_checker('abcnews.com')


In [7]:


from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

Options = Options()
Options.headless = True 

domain = 'ralblaw.com'
def check_login(domain, driver):
    wp_xpath = "//a[starts-with(@href, 'https://wordpress.org')]"
    try:
        displayed = driver.find_element('xpath', wp_xpath).is_displayed()

        return displayed
    except:

        return False

def check_page_source(domain, driver):
    try:
        driver.get(f"https://{domain}/wp-admin")
        wp_pg = "wp-content" in driver.page_source
        return wp_pg
    except:

        return False

def check_wp(domain):

    service_obj = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(options=Options, service=service_obj)
    driver.implicitly_wait(1)
    driver.get(f"https://{domain}/wp-admin")
    login = check_login(domain, driver)
    pg_source = check_page_source(domain, driver)
    wp = None
    if login == False and pg_source == False:
        wp = False
    else:
        wp = True

    driver.close()
    return wp



   



In [8]:
from urllib.parse import urlparse

with open('FilipinoFakeNews.txt', 'r') as f:
    fakenews_sites = f.read().split('\n')
    fakenews_sites = [site.strip() for site in fakenews_sites]

with open('FilipinoRealNews.txt', 'r') as f:
    realnews_sites = f.read().split('\n')
    realnews_sites = [site.strip() for site in realnews_sites]
realnews_sites = [urlparse(url).netloc.strip("www.") for url in realnews_sites]

site_dict = {'Fake': fakenews_sites, 'Real': realnews_sites}
site_dict


{'Fake': ['akoy-pilipino.blogspot.com',
  'aljazeeranews-tv.com',
  'asensopinoy.info',
  'balitaonline.xyz',
  'citizenexpress.today',
  'dailyartikulo.info',
  'dailyfilipino.altervista.org',
  'dailyinsights.today',
  'thedailysentry.net',
  'dailyviralhub.altervista.org',
  'dedma.com',
  'definitelyfilipino.com',
  'news.definitelyfilipino.net',
  'balita.definitelyfilipino.com',
  'buzz.definitelyfilipino.com',
  'buzz.definitelyfilipino.net',
  'dugongmaharlika.com',
  'du30newsblog.blogspot.com',
  'dutertetrendingnews.blogspot.com',
  'dutertepilipinas.info',
  'lugto.com',
  'France24-tv.com',
  'globalnews.favradio.fm',
  'goforwin.info',
  'goodnewsnetworkph.com',
  'goodnewstoday.d30.club',
  'gossipdiary.com',
  'hotnewsphil.blogspot.com',
  'www.iampilipino.com',
  'internationallatestupdates.blogspot.com',
  'kabalitaka.blogspot.com',
  'kantonewsph.xyz',
  'kantonewsph.info',
  'leaknewsph.com',
  'maharlikanews.com',
  'makibalita.info',
  'mediacurious.com',
  'media

In [9]:
from collections import defaultdict

from urllib.request import urlopen, URLError
import socket

def check_url( url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}
    response = requests.get('https://'+url, timeout=20, headers=headers)
    try:
        return response.status_code < 400 
    except URLError as e:
        return False
    except socket.timeout as e:
        print(False)


In [10]:
def get_site_data(domain, rating):
    try:
        if check_url(domain) == True:
            pr_int, pr_dec, rank = get_pagerank(domain)
            domain_registrar, postal_code, country = whois_checker(domain)
            harmless, malicious, suspicious, undetected, timeout = virus_scan(domain)
            local_n, outbound_links, total_links, links = link_checker(domain)
            wp_check = check_wp(domain)
            return domain, pr_int, pr_dec, rank, domain_registrar, postal_code, country, harmless, malicious, suspicious, undetected, timeout, local_n, outbound_links, total_links, links, wp_check, rating 
        else:
            return 'Site down'
    except:
        return 'Site down'

In [11]:




    

# domain = 'reddit.com'

columns = ['Domain', 'Page rank integer', 'Page rank decimal', 'Site Rank', 'Domain registrar', 'Postal code', 'Country of origin', 'Harmless', 'Malicious', 'Suspicious', 'Undetected', 'Timeout', 'No. of Local links', 'No. of Outbound links', 'Total links', 'Links to fake news site?', 'Wordpress?', 'Site Rating']

site_df_dict = defaultdict(list)

for rating, domains in site_dict.items():
    for i, domain in enumerate(domains):
        print(f'Analyzing Site {i+1}: {domain} of rating: {rating}')
        site_data = get_site_data(domain.lower(), rating)
        if type(site_data) == tuple:
            for column, data in zip(columns, site_data):
                if type(data) != list:
                    site_df_dict[column].append(data)
                else:
                    if any(site in data for site in fakenews_sites):
                        site_df_dict[column].append(True)
                    else:
                        site_df_dict[column].append(False)
            print('Data Gathered')
        else:
            print('site down')




Analyzing Site 1: akoy-pilipino.blogspot.com of rating: Fake
site down
Analyzing Site 2: aljazeeranews-tv.com of rating: Fake
site down
Analyzing Site 3: asensopinoy.info of rating: Fake
site down
Analyzing Site 4: balitaonline.xyz of rating: Fake


[WDM] - Downloading: 100%|██████████| 8.61M/8.61M [00:01<00:00, 5.21MB/s]


Data Gathered
Analyzing Site 5: citizenexpress.today of rating: Fake
Data Gathered
Analyzing Site 6: dailyartikulo.info of rating: Fake
site down
Analyzing Site 7: dailyfilipino.altervista.org of rating: Fake
site down
Analyzing Site 8: dailyinsights.today of rating: Fake
site down
Analyzing Site 9: thedailysentry.net of rating: Fake
Data Gathered
Analyzing Site 10: dailyviralhub.altervista.org of rating: Fake
site down
Analyzing Site 11: dedma.com of rating: Fake
Data Gathered
Analyzing Site 12: definitelyfilipino.com of rating: Fake
Data Gathered
Analyzing Site 13: news.definitelyfilipino.net of rating: Fake
Data Gathered
Analyzing Site 14: balita.definitelyfilipino.com of rating: Fake
Data Gathered
Analyzing Site 15: buzz.definitelyfilipino.com of rating: Fake
site down
Analyzing Site 16: buzz.definitelyfilipino.net of rating: Fake
Data Gathered
Analyzing Site 17: dugongmaharlika.com of rating: Fake
site down
Analyzing Site 18: du30newsblog.blogspot.com of rating: Fake
site down
Ana

In [12]:
site_df_dict

defaultdict(list,
            {'Domain': ['balitaonline.xyz',
              'citizenexpress.today',
              'thedailysentry.net',
              'dedma.com',
              'definitelyfilipino.com',
              'news.definitelyfilipino.net',
              'balita.definitelyfilipino.com',
              'buzz.definitelyfilipino.net',
              'kantonewsph.info',
              'nagbabagangbalita.info',
              'netcitizen.co',
              'newscenterph.info',
              'newsmediaph.com',
              'pinoytrending.altervista.org',
              'pinoytrendingnews.net',
              'pinoytrending.altervista.org',
              'pinoytrending.altervista.org',
              'sowhatsnews.wordpress.com',
              'tahonews.com',
              'thenewsfeeder.net',
              'tnp.today',
              'trendingbalita.info',
              'update.ph',
              'xolxol.ph',
              'businessmirror.com.ph',
              'tribune.net.ph',
             

In [14]:
import pandas as pd
site_df = pd.DataFrame(site_df_dict)
site_df


,Domain,Page rank integer,Page rank decimal,Site Rank,Domain registrar,Postal code,Country of origin,Harmless,Malicious,Suspicious,Undetected,Timeout,No. of Local links,No. of Outbound links,Total links,Links to fake news site?,Wordpress?,Site Rating
0,balitaonline.xyz,0,0.00,None,DYNADOT LLC,REDACTED FOR PRIVACY,False,78,1,0,8,0,0,0,0,False,False,Fake
1,citizenexpress.today,2,1.92,34163772,NAMECHEAP INC,False,IS,80,0,0,7,0,23,0,23,True,True,Fake
2,thedailysentry.net,3,3.28,3623897,NAMECHEAP INC,101,IS,78,0,0,9,0,26,64,90,False,False,Fake
3,dedma.com,0,0.00,None,NAMECHEAP INC,101,IS,79,0,0,8,0,11,0,11,True,False,Fake
4,definitelyfilipino.com,4,4.23,414332,"Domain.com, LLC",32256,US,79,0,0,8,0,0,3,3,False,False,Fake
5,news.definitelyfilipino.net,2,1.93,33880878,"Domain.com, LLC",32256,US,78,0,0,9,0,39,2,41,True,True,Fake
6,balita.definitelyfilipino.com,2,1.93,33801871,"Domain.com, LLC",32256,US,79,0,0,8,0,31,2,33,True,True,Fake
7,buzz.definitelyfilipino.net,3,2.71,8518353,"Domain.com, LLC",32256,US,78,0,0,9,0,51,1,52,True,True,Fake
8,kantonewsph.info,0,0.00,None,DYNADOT LLC,REDACTED FOR PRIVACY,False,82,0,0,6,0,0,1,1,False,False,Fake
9,nagbabagangbalita.info,0,0.00,None,DYNADOT LLC,REDACTED FOR PRIVACY,False,81,0,0,7,0,0,1,1,False,False,Fake


In [17]:
site_df.to_csv('FakeReal_News_Sites.csv', mode='a', index=True, header=False)


In [ ]:
# def check_data(domain):
#     pr_int, pr_dec, rank = get_pagerank(domain)
#     domain_registrar, postal_code, country = whois_checker(domain)
#     harmless, malicious, suspicious, undetected, timeout = virus_scan(domain)
#     local_n, outbound_links, total_links, links = link_checker(domain)
#     wp_check = check_wp(domain)
# check_data('21stCenturyWire.com')